Aravind Patnam
Jeremy Tan
Timothy Le

CSE-184 Final Project - Trending Here Trending There 
An analysis on trending and nontrending Youtube videos. 

In [ ]:
import scripts.scripts as script
%run "scripts/imports.py"
import plotly.io as pio
pio.renderers.default = 'iframe'

In [ ]:
## written by Aravind Patnam and Jeremy Tan
#### get all csv dataframes for trending. These files should be in the same directory. 
US_trending_df = pd.read_csv('data/USvideos.csv') #USA
CA_trending_df = pd.read_csv('data/CAvideos.csv') #CANADA
DE_trending_df = pd.read_csv('data/DEvideos.csv') #GERMANY
FR_trending_df = pd.read_csv('data/FRvideos.csv') #FRANCE
GB_trending_df = pd.read_csv('data/GBvideos.csv') #GREAT BRITAIN
IN_trending_df = pd.read_csv('data/INvideos.csv') #INDIA
JP_trending_df = pd.read_csv('data/JPvideos.csv', encoding='ISO-8859-1') #JAPAN
KR_trending_df = pd.read_csv('data/KRvideos.csv' , encoding='ISO-8859-1') #SOUTH KOREA
MX_trending_df = pd.read_csv('data/MXvideos.csv', encoding='ISO-8859-1') #MEXICO
RU_trending_df = pd.read_csv('data/RUvideos.csv', encoding='ISO-8859-1') #RUSSIA

list_of_all_trending_dfs = [US_trending_df, CA_trending_df, DE_trending_df, FR_trending_df, GB_trending_df, IN_trending_df,
                           JP_trending_df, KR_trending_df, MX_trending_df, RU_trending_df]
list_of_csvs = ['data/USvideos.csv','data/CAvideos.csv', 'data/DEvideos.csv', 'data/FRvideos.csv', 'data/GBvideos.csv', 'data/INvideos.csv', 'data/JPvideos.csv','data/KRvideos.csv', 'data/MXvideos.csv', 'data/RUvideos.csv' ]

big_df = list()
for csv in list_of_csvs:
    # use encoding to bypass utf error
    df = pd.read_csv(csv, index_col='video_id', encoding='ISO-8859-1')
    # add new column called "country" to indentify which videos the csv are coming from
    # depending on your path name, this will break as it looks at the path name 
    df['country'] = csv[5:7]
    big_df.append(df)
    
full_trending_df = pd.concat(big_df)
full_trending_df.tail()

In [ ]:
#### reformatting and detecting nans

# reformat trending_date
full_trending_df['trending_date'] = pd.to_datetime(full_trending_df['trending_date'],errors='coerce', format='%y.%d.%m')
full_trending_df['publish_time'] = pd.to_datetime(full_trending_df['publish_time'], errors='coerce', format='%Y-%m-%dT%H:%M:%S.%fZ')

# detects any nans 
full_trending_df = full_trending_df[full_trending_df['trending_date'].notnull()]
full_trending_df = full_trending_df[full_trending_df['publish_time'].notnull()]

# drop all nans by trmeoving them 
full_trending_df = full_trending_df.dropna(how='any',inplace=False, axis = 0)

# this is done already so don't run it twice
full_trending_df.insert(4, 'publish_date', full_trending_df['publish_time'].dt.date)
full_trending_df['publish_time'] = full_trending_df['publish_time'].dt.time

# set index by video id and sort by trending dates 
full_trending_df_fill = full_trending_df.reset_index().sort_values('trending_date').set_index('video_id')
# set index by vide id and sort by trending dates, but make sure to drop duplicates
full_trending_df = full_trending_df.reset_index().sort_values('trending_date').drop_duplicates('video_id',keep='last').set_index('video_id')
# prep data to by adding like rate and spliitng publish time into a hour, min, and sec column
full_trending_df['like_rate'] =  full_trending_df ['likes'] / full_trending_df['views'] * 100
full_trending_df[['hour','min','sec']] = full_trending_df['publish_time'].astype(str).str.split(':', expand=True).astype(int)
full_trending_df.head()

In [ ]:
## written by Aravind Patnam

## run these to get the non trending datasets generated from the youtube api. These files should be in the same directory
not_trending_us_df = pd.read_csv('data/not_trending_us_df.csv')
not_trending_ca_df = pd.read_csv('data/not_trending_ca_df.csv')
not_trending_de_df = pd.read_csv('data/not_trending_de_df.csv')
not_trending_fr_df = pd.read_csv('data/not_trending_fr_df.csv')
not_trending_gb_df = pd.read_csv('data/not_trending_gb_df.csv')
not_trending_in_df = pd.read_csv('data/not_trending_in_df.csv')
not_trending_jp_df = pd.read_csv('data/not_trending_jp_df.csv')
not_trending_kr_df = pd.read_csv('data/not_trending_kr_df.csv')
not_trending_mx_df = pd.read_csv('data/not_trending_mx_df.csv')
not_trending_ru_df = pd.read_csv('data/not_trending_ru_df.csv')

In [ ]:
## written by Aravind Patnam
### do not run this unless you have a lot of time and 10 Youtube API keys!!!. Datasets are already generated for you for testing. 
n = 50
US_trending_videoIds = US_trending_df.sample(n)['video_id'].tolist()
CA_trending_videoIds = CA_trending_df.sample(n)['video_id'].tolist()
DE_trending_videoIds = DE_trending_df.sample(n)['video_id'].tolist()
FR_trending_videoIds = FR_trending_df.sample(n)['video_id'].tolist()
GB_trending_videoIds = GB_trending_df.sample(n)['video_id'].tolist()
IN_trending_videoIds = IN_trending_df.sample(n)['video_id'].tolist()
JP_trending_videoIds = JP_trending_df.sample(n)['video_id'].tolist()
KR_trending_videoIds = KR_trending_df.sample(n)['video_id'].tolist()
MX_trending_videoIds = MX_trending_df.sample(n)['video_id'].tolist()
RU_trending_videoIds = RU_trending_df.sample(n)['video_id'].tolist()

## written by Aravind Patnam
### do not run this unless you have a lot of time and 10 Youtube API keys!!! Datasets are already generated for you for testing. 
## do following requests separately with a new API Key and have the file called "apiKey"
not_trending_us_df = script.process_youtube_requests(US_trending_videoIds)
not_trending_ca_df = script.process_youtube_requests(CA_trending_videoIds)
not_trending_de_df = script.process_youtube_requests(DE_trending_videoIds)
not_trending_fr_df = script.process_youtube_requests(FR_trending_videoIds)
not_trending_gb_df = script.process_youtube_requests(GB_trending_videoIds)
not_trending_in_df = script.process_youtube_requests(IN_trending_videoIds)
not_trending_jp_df = script.process_youtube_requests(JP_trending_videoIds)
not_trending_kr_df = script.process_youtube_requests(KR_trending_videoIds)
not_trending_mx_df = script.process_youtube_requests(MX_trending_videoIds)
not_trending_ru_df = script.process_youtube_requests(RU_trending_videoIds)

In [ ]:
## written by Aravind Patnam

## puts all nontrending and all dfs that we have together for one big df and multiple smaller ones
list_of_all_nontrending_dfs = [not_trending_us_df, not_trending_ca_df, not_trending_de_df, 
                              not_trending_fr_df, not_trending_gb_df, not_trending_in_df, not_trending_jp_df,
                              not_trending_kr_df, not_trending_mx_df, not_trending_ru_df]


# aravind, the categories are already in the csv??????????????????
full_nontrending_df = pd.concat(list_of_all_nontrending_dfs)
allDfsList = [full_trending_df] + list_of_all_trending_dfs
allDfsDf = pd.concat(allDfsList)
allDfsList.append(allDfsDf)

## written by Jeremy Tan
## insert new category field into dataframes
for df in allDfsList:
    script.insert_category_field(df)
 
    
## written by Jeremy Tan
## converts all columns in dataframes to type for analysis
for df in list_of_all_nontrending_dfs:
    df['video_id'] = df['video_id'].astype(str) 
    df['title'] = df['title'].astype(str)
    df['channel_title'] = df['channel_title'].astype(str)
    df['category_id'] = df['category_id'].astype(int)
    #df['category'] = df['category'].astype(str)
    df['tags'] = df['tags'].astype(str)
    df['views'] = df['views'].astype(int)
    df['likes'] = df['likes'].astype(int)
    df['dislikes'] = df['dislikes'].astype(int)
    df['comment_count'] = df['comment_count'].astype(int)
    df['description'] = df['description'].astype(str)

In [ ]:
full_nontrending_df.head()

In [ ]:
## written by Aravind Patnam
full_nontrending_df = pd.concat(list_of_all_nontrending_dfs)
allDfsList = list_of_all_trending_dfs + list_of_all_nontrending_dfs + [full_trending_df] + [full_nontrending_df]
allDfsDf = pd.concat(allDfsList)
allDfsList.append(allDfsDf)

## output is a map containing all the numeric data for each df
describes = []
for df in allDfsList:
    describes.append(script.find_stats(df))
describesKeys = ['US_trending_df', 'CA_trending_df', 'DE_trending_df', 'FR_trending_df', 'GB_trending_df',
                'IN_trending_df', 'JP_trending_df', 'KR_trending_df', 'MX_trending_df', 'RU_trending_df',
                 'not_trending_us_df', 'not_trending_ca_df', 'not_trending_de_df', 'not_trending_fr_df',
                'not_trending_gb_df', 'not_trending_in_df', 'not_trending_jp_df', 'not_trending_kr_df',
                 'not_trending_mx_df', 'not_trending_ru_df',
                'full_trending_df', 'full_nontrending_df', 'allDfsDf']
describeMap = dict(zip(describesKeys, describes))
## make sense of each data point we have from the describeMap and clean it for visualization
countries = list(describeMap.keys())
print(countries)
## written by Aravind Patnam
likes_count, dislikes_count = script.do_describe_analysis(describeMap, countries, describesKeys)

In [ ]:
script.do_likes_to_dislikes_analysis(likes_count, dislikes_count, countries)

In [ ]:
## written by Aravind Patnam

## a bunch of maps containing the most popular tags for each country that are both trending and not trending and their frequency
us_trending_most_common_tags = script.get_most_common_tags(US_trending_df)
ca_trending_most_common_tags = script.get_most_common_tags(CA_trending_df)
de_trending_most_common_tags = script.get_most_common_tags(DE_trending_df)
fr_trending_most_common_tags = script.get_most_common_tags(FR_trending_df)
gb_trending_most_common_tags = script.get_most_common_tags(GB_trending_df)
in_trending_most_common_tags = script.get_most_common_tags(IN_trending_df)
jp_trending_most_common_tags = script.get_most_common_tags(JP_trending_df)
kr_trending_most_common_tags = script.get_most_common_tags(KR_trending_df)
mx_trending_most_common_tags = script.get_most_common_tags(MX_trending_df)
ru_trending_most_common_tags = script.get_most_common_tags(RU_trending_df)

us_nontrending_most_common_tags = script.get_most_common_tags(not_trending_us_df)
ca_nontrending_most_common_tags = script.get_most_common_tags(not_trending_ca_df)
de_nontrending_most_common_tags = script.get_most_common_tags(not_trending_de_df)
fr_nontrending_most_common_tags = script.get_most_common_tags(not_trending_fr_df)
gb_nontrending_most_common_tags = script.get_most_common_tags(not_trending_gb_df)
in_nontrending_most_common_tags = script.get_most_common_tags(not_trending_in_df)
jp_nontrending_most_common_tags = script.get_most_common_tags(not_trending_jp_df)
kr_nontrending_most_common_tags = script.get_most_common_tags(not_trending_kr_df)
mx_nontrending_most_common_tags = script.get_most_common_tags(not_trending_mx_df)
ru_nontrending_most_common_tags = script.get_most_common_tags(not_trending_ru_df)

Below is a wordcloud visualization of some of the countries and a comparison of them between trending and non-trending videos. Since there is not much space to do all 20 different combinations, only the 3 with the most popular tags were displayed. 

In [ ]:
## written by Aravind Patnam
script.do_wordcloud(us_trending_most_common_tags, gb_trending_most_common_tags, in_trending_most_common_tags, us_nontrending_most_common_tags, gb_nontrending_most_common_tags, in_nontrending_most_common_tags)

Below represents a sentiment analysis conducted on the youtube tags. A datset of positive words and negative words is provided for the model to work. 

In [ ]:
## written by Aravind Patnam

## gets the classifications from the sentiment analysis and prints out accuracies of the model
classifications_us_trending = script.execute_model(list(us_trending_most_common_tags.keys()))
classifications_ca_trending = script.execute_model(list(ca_trending_most_common_tags.keys()))
classifications_de_trending = script.execute_model(list(de_trending_most_common_tags.keys()))
classifications_fr_trending = script.execute_model(list(fr_trending_most_common_tags.keys()))
classifications_gb_trending = script.execute_model(list(gb_trending_most_common_tags.keys()))
classifications_in_trending = script.execute_model(list(in_trending_most_common_tags.keys()))
classifications_jp_trending = script.execute_model(list(jp_trending_most_common_tags.keys()))
classifications_kr_trending = script.execute_model(list(kr_trending_most_common_tags.keys()))
classifications_mx_trending = script.execute_model(list(mx_trending_most_common_tags.keys()))
classifications_ru_trending = script.execute_model(list(ru_trending_most_common_tags.keys()))

classifications_us_nontrending = script.execute_model(list(us_nontrending_most_common_tags.keys()))
classifications_ca_nontrending = script.execute_model(list(ca_nontrending_most_common_tags.keys()))
classifications_de_nontrending = script.execute_model(list(de_nontrending_most_common_tags.keys()))
classifications_fr_nontrending = script.execute_model(list(fr_nontrending_most_common_tags.keys()))
classifications_gb_nontrending = script.execute_model(list(gb_nontrending_most_common_tags.keys()))
classifications_in_nontrending = script.execute_model(list(in_nontrending_most_common_tags.keys()))
classifications_jp_nontrending = script.execute_model(list(jp_nontrending_most_common_tags.keys()))
classifications_kr_nontrending = script.execute_model(list(kr_nontrending_most_common_tags.keys()))
classifications_mx_nontrending = script.execute_model(list(mx_nontrending_most_common_tags.keys()))
classifications_ru_nontrending = script.execute_model(list(ru_nontrending_most_common_tags.keys()))

In [ ]:
## written by Aravind Patnam

## calls above method for stats for each country for visualization

country_us_trending, us_trending_pos, us_trending_neg = script.get_sentiment_stats(classifications_us_trending, "USA_Trending")
country_ca_trending, ca_trending_pos, ca_trending_neg = script.get_sentiment_stats(classifications_ca_trending, "Canada_Trending")
country_de_trending, de_trending_pos, de_trending_neg = script.get_sentiment_stats(classifications_de_trending, "Denmark_Trending")
country_fr_trending, fr_trending_pos, fr_trending_neg = script.get_sentiment_stats(classifications_fr_trending, "France_Trending")
country_gb_trending, gb_trending_pos, gb_trending_neg = script.get_sentiment_stats(classifications_gb_trending, "GreatBritain_Trending")
country_in_trending, in_trending_pos, in_trending_neg = script.get_sentiment_stats(classifications_in_trending, "India_Trending")
country_jp_trending, jp_trending_pos, jp_trending_neg = script.get_sentiment_stats(classifications_jp_trending, "Japan_Trending")
country_kr_trending, kr_trending_pos, kr_trending_neg = script.get_sentiment_stats(classifications_kr_trending, "SouthKorea_Trending")
country_mx_trending, mx_trending_pos, mx_trending_neg = script.get_sentiment_stats(classifications_mx_trending, "Mexico_Trending")
country_ru_trending, ru_trending_pos, ru_trending_neg = script.get_sentiment_stats(classifications_ru_trending, "Russia_Trending")


country_us_nontrending, us_nontrending_pos, us_nontrending_neg = script.get_sentiment_stats(classifications_us_nontrending, "USA_NonTrending")
country_ca_nontrending, ca_nontrending_pos, ca_nontrending_neg = script.get_sentiment_stats(classifications_ca_nontrending, "Canada_NonTrending")
country_de_nontrending, de_nontrending_pos, de_nontrending_neg = script.get_sentiment_stats(classifications_de_nontrending, "Denmark_NonTrending")
country_fr_nontrending, fr_nontrending_pos, fr_nontrending_neg = script.get_sentiment_stats(classifications_fr_nontrending, "France_NonTrending")
country_gb_nontrending, gb_nontrending_pos, gb_nontrending_neg = script.get_sentiment_stats(classifications_gb_nontrending, "GreatBritain_NonTrending")
country_in_nontrending, in_nontrending_pos, in_nontrending_neg = script.get_sentiment_stats(classifications_in_nontrending, "India_NonTrending")
country_jp_nontrending, jp_nontrending_pos, jp_nontrending_neg = script.get_sentiment_stats(classifications_jp_nontrending, "Japan_NonTrending")
country_kr_nontrending, kr_nontrending_pos, kr_nontrending_neg = script.get_sentiment_stats(classifications_kr_nontrending, "SouthKorea_NonTrending")
country_mx_nontrending, mx_nontrending_pos, mx_nontrending_neg = script.get_sentiment_stats(classifications_mx_nontrending, "Mexico_NonTrending")
country_ru_nontrending, ru_nontrending_pos, ru_nontrending_neg = script.get_sentiment_stats(classifications_ru_nontrending, "Russia_NonTrending")

countries = [country_us_trending, country_ca_trending, country_de_trending, country_fr_trending, country_gb_trending,
            country_in_trending, country_jp_trending, country_kr_trending, country_mx_trending, country_ru_trending,
            country_us_nontrending, country_ca_nontrending, country_de_nontrending, country_fr_nontrending, 
            country_gb_nontrending, country_in_nontrending, country_jp_nontrending, country_kr_nontrending, 
            country_mx_nontrending, country_ru_nontrending]
positivePercenteages = [us_trending_pos, ca_trending_pos, de_trending_pos, fr_trending_pos, gb_trending_pos, in_trending_pos, jp_trending_pos,
            kr_trending_pos, mx_trending_pos, ru_trending_pos, us_nontrending_pos, ca_nontrending_pos, de_nontrending_pos, fr_nontrending_pos,
            gb_nontrending_pos, in_nontrending_pos, jp_nontrending_pos, kr_nontrending_pos, mx_nontrending_pos, ru_nontrending_pos]
negativePercentages = [us_trending_neg, ca_trending_neg, de_trending_neg, fr_trending_neg, gb_trending_neg, in_trending_neg, jp_trending_neg, 
                      kr_trending_neg, mx_trending_neg, ru_trending_neg, us_nontrending_neg, ca_nontrending_neg, de_nontrending_neg, 
                      fr_nontrending_neg, gb_nontrending_neg, in_nontrending_neg, jp_nontrending_neg, kr_nontrending_neg, mx_nontrending_neg,
                      ru_nontrending_neg]
posToNegRatios = [i / j for i, j in zip(positivePercenteages, negativePercentages)] 
data_dict = {"Country" : countries, "Positives": positivePercenteages, "Negatives": negativePercentages, "PositiveNegativeRatio": posToNegRatios}
data = pd.DataFrame(data_dict, columns = ['Country', 'Positives' , 'Negatives', 'PositiveNegativeRatio'])

data

Hover over the plots and switch table to play with them and see what each one represents!! Use the toolbar to crop out some plots for better analysis. The first plot visualizes the ratios between the positive and negative tags while the second one shows the actual values that were classified as positive and negative by the sentiment analysis classifier. 

In [ ]:
## written by Aravind Patnam
script.do_sentiment_analysis_visualization(data)

Hover and play around with the principle component analysis presented below. LDA topic model was used on youtube tags and descriptions to show these findings. The relevancy metric at the time of this submission is set to around 0.30 since it showed the most favorable results that were not too specific and not too generic. 

In [ ]:
## written by Aravind Patnam

## this will take a long time to run!

## visualize trending videos using pyLDAvis -> this might be only visible on nbviewer depending on your notebook viewing settings

full_trending_lda_input = list(script.get_most_common_tags(full_trending_df).keys()) + list (full_trending_df.sample(16901)['description'])
topics_Full_Trending, corpus, dictionary = script.do_LDA(full_trending_lda_input)
lda_display = script.visualize_LDA(True, corpus, dictionary)
pyLDAvis.display(lda_display)

In [ ]:
## written by Aravind Patnam

## this will take a long time to run!

## visualize nontrending videos using pyLDAvis -> this might be only visible on nbviewer depending on your notebook viewing settings


full_nontrending_lda_input = list(script.get_most_common_tags(full_nontrending_df).keys()) + list (full_nontrending_df['description'])
topics_Full_Nontrending, corpus, dictionary = script.do_LDA(full_nontrending_lda_input)
lda_display = script.visualize_LDA(True, corpus, dictionary)
pyLDAvis.display(lda_display)

In [ ]:
## written by Aravind Patnam
 
## this will take a long time to run!

## visualize all videos using pyLDAvis -> this might be only visible on nbviewer depending on your notebook viewing settings

allDfsDf_lda_input = list(script.get_most_common_tags(allDfsDf).keys()) + list (allDfsDf.sample(100000)['description'])
topics_all_dfs, corpus, dictionary = script.do_LDA(allDfsDf_lda_input)
lda_display = script.visualize_LDA(True, corpus, dictionary)
pyLDAvis.display(lda_display)

In [ ]:
## written by Aravind Patnam
## no need to run this as this has already been done for you
## convert to csvs

not_trending_us_df.to_csv('data/not_trending_us_df.csv')
not_trending_ca_df.to_csv('data/not_trending_ca_df.csv')
not_trending_de_df.to_csv('data/not_trending_de_df.csv')
not_trending_fr_df.to_csv('data/not_trending_fr_df.csv')
not_trending_gb_df.to_csv('data/not_trending_gb_df.csv')
not_trending_in_df.to_csv('data/not_trending_in_df.csv')
not_trending_jp_df.to_csv('data/not_trending_jp_df.csv')
not_trending_kr_df.to_csv('data/not_trending_kr_df.csv')
not_trending_mx_df.to_csv('data/not_trending_mx_df.csv')
not_trending_ru_df.to_csv('data/not_trending_ru_df.csv')

In [ ]:
# Written by Jeremy Tan
script.makeVideoTrending(full_trending_df, full_trending_df_fill)

As you can see from the visualization above, most trending videos only stay trending for two days or less. Some notable outliers are videos from Great Britan, which has videos that will stay trending for close to 30 days! For countries in the far East, videos only seem to stay tredning for one day so the turnover rate is pretty high.

In [ ]:
# Written by Jeremy Tan 
# Makes two new columns that count tilte length and description length ands changes comment_count to comments
full_trending_df['title_length'] = full_trending_df['title'].str.len()
full_trending_df['description_length'] = full_trending_df['description'].str.len()
full_trending_df = full_trending_df.rename(columns={'comment_count':'comments'})

In [ ]:
# Written by Jeremy Tan
# What is the correlation between views, likes, disklies, and comment count?
script.makeScatter(full_trending_df)

As seen from the scatter matrix, there is a somewhat strong correlation between views and likes. Another correlation is between likes and comments, views and comments, and dislikes and comments (though somewhat weak!). Another interesting point is the length of the description and the length of the title seem to be tied together.

In [ ]:
# What is the correlation between views, likes, disklies, and comment count?
# Another interactive visualization
corr = full_trending_df.loc[:, ['views', 'likes', 'dislikes', 'comments', 'description_length', 'title_length']].corr()
script.makeHeatMap(corr)

Another way to visualzie the correlations. Here, I can see additional correlations of dislikes and views, dislikes and likes, and dislikes and comments.

In [ ]:
# Written by Jeremy Tan
# What is the correlation between views, likes, disklies, and comment count in categories?
categories = full_trending_df['category'].unique()                                                                                                                                                     
corr_list = [full_trending_df[full_trending_df['category'] == cat].loc[:, ['views', 'likes', 'dislikes', 'comments', 'description_length', 'title_length']].corr() for cat in categories] 
script.makeCategoryHeatMap(corr_list, categories)

Another way to visualize correlations but now based on categories. For the popular categories, the correlations I stated earlier hold true. However, if you were to go to a unpopular category like "trailers," negative correlations appear in place of the previous, strong correlations. Most suprisingly, for categories that elict human emotion, such as "Pet & Animals" and "Nonprofits & Activism" there is a strong correlation between likes, views, comments, and dislikes. 

In [ ]:
# Written by Jeremy Tan
# What category of videos trend the most in which countries?
# What category of videos fail to hit trending in which countries?

In [ ]:
script.trendingCategories(US_trending_df, "United States")

In [ ]:
script.nontrendingCategories(not_trending_us_df, "United States")

Based on the two plot above, one can see "Entertainment" videos are the majority of videos made. They, however, have the highest chance to both fail and succeed. The pattern illuminates the more successful the category, the more videos that are being put out.  

In [ ]:
script.trendingCategories(CA_trending_df, "Canada")

In [ ]:
script.nontrendingCategories(not_trending_ca_df, "Canada")

The same pattern seems to appear as it did in the United States. However, "People and Blogs" and "Music" seem to fail more in this country.

In [ ]:
script.trendingCategories(DE_trending_df, "Germany")

In [ ]:
script.nontrendingCategories(not_trending_de_df, "Germany")

The same pattern seems to appear as it did in the United States. However, "People and Blogs" have more of a success rate but "Films & Animation" seem to fail more.

In [ ]:
script.trendingCategories(FR_trending_df, "France")

In [ ]:
script.nontrendingCategories(not_trending_fr_df, "France")

Same pattern occurs like in the Untied States. An interesting point is that "Peoples & Blog" have a higher probability of being trending. We can assume this is due to people valuing these type of videos more.

In [ ]:
script.trendingCategories(GB_trending_df, "Great Britan")

In [ ]:
script.nontrendingCategories(not_trending_gb_df, "Great Britan")

Same pattern occurs like in the Untied States. However, "Music" dominates this country with the majority of the videos being produced are of this category.

In [ ]:
script.trendingCategories(IN_trending_df, "India")

In [ ]:
script.nontrendingCategories(not_trending_in_df, "India")

Same pattern as the US. Nothing interesitng to note as it follows the pattern exactly where the more successful a category, the more likely the video will also fail.

In [ ]:
script.trendingCategories(JP_trending_df, "Japan")

In [ ]:
script.nontrendingCategories(not_trending_jp_df, "Japan")

Similar pattern as Germany, where "People and Blogs" videos will have a higher chance to be tredning and not fail.

In [ ]:
script.trendingCategories(KR_trending_df, "Korea")

In [ ]:
script.nontrendingCategories(not_trending_kr_df, "Korea")

Same pattern as the US.Interesting enough, this country cares alot about "News & Politics" and have lots of video that hit trending (only second behind "Entertainment"). 

In [ ]:
script.trendingCategories(MX_trending_df, "Mexico")

In [ ]:
script.nontrendingCategories(not_trending_mx_df, "Mexico")

Same pattern as the US. Similar stats as the US too. 

In [ ]:
script.trendingCategories(RU_trending_df, "Russia")

In [ ]:
script.nontrendingCategories(not_trending_ru_df, "Russia")

Same pattern as the US. However, like Germany and Japan, "Peoples & Blogs" have a high succes rate to reach trending and fail less.

In [ ]:
# Written by Jeremy Tan
# Which categories are the most popular? Do highest average amongst likes and views ----> shows which type of videos people enjoy the most
# makes a box plot to visuzlaize distbution of likes and views 

In [ ]:
script.likes_to_categories(US_trending_df, "United States", 'likes_log')

In [ ]:
script.likes_to_categories(US_trending_df, "United States", 'views_log')

Judging from the boxplot, it seems people in the US enjoy videos categorized Music more the most. Gaming is the second most popualr. Then, Entertainment. 

In [ ]:
script.likes_to_categories(CA_trending_df, "Canada", 'likes_log')

In [ ]:
script.likes_to_categories(CA_trending_df, "Canada", 'views_log')

It isn't as clear, but there seems to be a tie between Movies and Music as some of the more popular videos in Canada. Comedy lags behind third. 

In [ ]:
script.likes_to_categories(DE_trending_df, "Germany", 'likes_log')

In [ ]:
script.likes_to_categories(DE_trending_df, "Germany", 'views_log')

In Germany, Music is the most popular category. Movies trail right behind, and Comedy right after.

In [ ]:
script.likes_to_categories(FR_trending_df, "France", 'likes_log')

In [ ]:
script.likes_to_categories(FR_trending_df, "France", 'views_log')

In France, Music is the most popular category like Germany. However, Comedy and Entertainment trail right behind.

In [ ]:
script.likes_to_categories(GB_trending_df, "Great Britan", 'likes_log')

In [ ]:
script.likes_to_categories(GB_trending_df, "Great Britan", 'views_log')

Music is the most popular category, while Nonprfits & Activism then Entertainment trail behind.

In [ ]:
script.likes_to_categories(IN_trending_df, "India", 'likes_log')

In [ ]:
script.likes_to_categories(IN_trending_df, "India", 'views_log')

It isn't clear which videos come out on top, but based on likes, Pets & Animals, Gaming, and then Comedy are the top categories.

In [ ]:
script.likes_to_categories(JP_trending_df, "Japan", 'likes_log')

In [ ]:
script.likes_to_categories(JP_trending_df, "Japan", 'views_log')

Funny enough, Science & Technology are the most popular videos follwoed by Music and then Comedy.

In [ ]:
script.likes_to_categories(KR_trending_df, "Korea", 'likes_log')

In [ ]:
script.likes_to_categories(KR_trending_df, "Korea", 'views_log')

Similar to Japan, Science & Technology is first as Musis is second. However, Sports is third. 

In [ ]:
script.likes_to_categories(MX_trending_df, "Mexico", 'likes_log')

In [ ]:
script.likes_to_categories(MX_trending_df, "Mexico", 'views_log')

Music is first, Gaming is second, and Comedy is third.

In [ ]:
script.likes_to_categories(RU_trending_df, "Russia", 'likes_log')

In [ ]:
script.likes_to_categories(RU_trending_df, "Russia", 'views_log')

Music is first Science and Technology is second. Comedy is third. 

In [ ]:
# Written by Jeremy Tan
# Which channels are the most successfucl at reaching trending?
# Grabs most reoccuring videos in trending 

In [ ]:
script.videos_top(US_trending_df, "United States")

In [ ]:
script.videos_top(CA_trending_df, "Canada")

In [ ]:
script.videos_top(DE_trending_df, "Germany")

In [ ]:
script.videos_top(FR_trending_df, "France")

In [ ]:
script.videos_top(GB_trending_df, "Great Britan")

In [ ]:
script.videos_top(IN_trending_df, "India")

In [ ]:
script.videos_top(JP_trending_df, "Japan")

Original scrapper did not properly encode characters correctly. Hence the weird symbols.

In [ ]:
script.videos_top(KR_trending_df, "Korea")

Original scrapper did not properly encode characters correctly. Hence the weird symbols.

In [ ]:
script.videos_top(MX_trending_df, "Mexico")

In [ ]:
script.videos_top(RU_trending_df, "Russia")

Original scrapper did not properly encode characters correctly. Hence the weird symbols.

In [ ]:
# Written by Jeremy Tan
# When a video gets published, what is the intial like rate that got them to trending?
# Has two plots: one shows which hour a video is most commonly published and the other is a boxen plot that shows differnet quartiles of the like_rate + outliers
script.showHours(full_trending_df)

It seems most trending videos are published at 4 pm. However, more engagemnt in terms of like happens at 8pm.

In [ ]:
full_nontrending_df.head()

In [ ]:
# Written by Jeremy Tan 
# Grabs the videos that have the most views, likes, or dislikes
# What videos have the most views, likes, and dislikes in the tredning dataset and nontrending dataset?

In [ ]:
script.visualize_most(full_trending_df, "views")

In [ ]:
script.visualize_most(full_nontrending_df, "views")

In [ ]:
script.visualize_most(full_trending_df, "likes")

In [ ]:
script.visualize_most(full_nontrending_df, "likes")

In [ ]:
script.visualize_most(full_trending_df, "dislikes")

In [ ]:
script.visualize_most(full_nontrending_df, "dislikes")

In [ ]:
# Written by Jeremy Tan
# Which country has the most active participation and engagemet?
script.engagement(full_trending_df)

Overall, Great Britan has the most active audience with the US seriously lagging behind.